In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel
from typing import Dict

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
llma_api_key = os.getenv('LLAMA_API_KEY')

In [4]:
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
LLMA_BASE_URL = "http://localhost:11434/v1"

In [5]:
deepseek_model_name = 'deepseek-chat'
llma_model_name = 'llama3.2'

In [6]:
deepseek_client = AsyncOpenAI(
    base_url = DEEPSEEK_BASE_URL, 
    api_key = deepseek_api_key
)

llma_client = AsyncOpenAI(
    base_url = LLMA_BASE_URL,
    api_key = llma_api_key
)

In [7]:
deepseek_model = OpenAIChatCompletionsModel(model=deepseek_model_name, openai_client=deepseek_client)
llma_model = OpenAIChatCompletionsModel(model=llma_model_name, openai_client=llma_client)

In [8]:
instructions1 = "You are witty sales agent for PKAI. You write professional and mindful emails"
instructions2 = "You are humorous sales agent for PKAI. You write professional and likely to get response emails"
instructions3 = "You are busy sales agent for PKAI. You write professional and concise emails"
agent1 = Agent(name="Agent 1", instructions=instructions1, model="gpt-4o-mini")
agent2 = Agent(name="Agent 2", instructions=instructions2, model=deepseek_model)
agent3 = Agent(name="Agent 3", instructions=instructions3, model=llma_model)

In [9]:
agent1_tool = agent1.as_tool(tool_name='agent_1_tool', tool_description='Write a cold email')
agent2_tool = agent2.as_tool(tool_name='agent_2_tool', tool_description='Write a cold email')
agent3_tool = agent3.as_tool(tool_name='agent_3_tool', tool_description='Write a cold email')

In [10]:
@function_tool
def send_email(subject:str, html_body:str) -> Dict[str, str]:
    sg_client = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("hello@priyanshukhandelwal.com")
    to_email = To("udemy@priyanshukhandelwal.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg_client.send(mail)
    print(f"Email sent with status code: {response.status_code}")
    return {'status':'success'}

In [11]:
subject_writer_instructions = '''
You are a subject writer for PKAI. You write professional and likely to get response subject based on the email content
'''
html_writer_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer_agent = Agent(name="Subject Writer", instructions=subject_writer_instructions, model="gpt-4o-mini")
html_writer_agent = Agent(name="HTML Writer", instructions=html_writer_instructions, model="gpt-4o-mini")

subject_writer_tool = subject_writer_agent.as_tool(tool_name='subject_writer_tool', tool_description='Write a subject')
html_writer_tool = html_writer_agent.as_tool(tool_name='html_writer_tool', tool_description='Write an HTML email body')

send_email_tool = send_email

In [12]:
email_tools = [ subject_writer_tool, html_writer_tool, send_email_tool]
content_tools = [agent1_tool, agent2_tool, agent3_tool]

In [13]:
emailer_instructions = '''
You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body.
'''

emailer_agent = Agent(name="Emailer", 
                      instructions=emailer_instructions, 
                      model="gpt-4o-mini",
                      tools=email_tools, 
                      handoff_description="Convert email to html and send email")


In [14]:
handoffs_for_sales_manager = [emailer_agent] 
# now command goes to emailer agent, so if anything goes wrong we have to pick emailer agent for it.


In [15]:
sales_manager_instructions = '''
You are a salesmamager working for PKAI. You use the tools give to you to generate cold emails.
You never generate sales emails by yourself; you always use tools.
You try all 3 agent_tools once before choosing the best one. You can use tools iteratively untill you find a best email to send.
You pick single best email using your own judgement of which email will be more effective.
Then you handoff to EMAILER agent to send the email.
'''

sales_manager_agent = Agent(name = "Sales Manager",
instructions = sales_manager_instructions,
model = "gpt-4o-mini",
tools = content_tools,
handoffs = handoffs_for_sales_manager
)

with trace("Automated SDR"):
    response = await Runner.run(sales_manager_agent, "Write and send a cold email to Dear Ceo of Legtis.com from Helena")

Email sent with status code: 202


A guardrail in an AI agent context refers to safety mechanisms and constraints designed to ensure the agent operates within acceptable boundaries and behaves reliably. These are protective measures that prevent the agent from taking harmful, inappropriate, or unintended actions.
Key types of guardrails include:
 - Input guardrails filter and validate what information or commands the agent receives, blocking malicious prompts, inappropriate content, or requests outside its intended scope.
 - Output guardrails monitor and filter the agent's responses before they're delivered, ensuring they don't contain harmful, biased, or inappropriate content.
 - Behavioral guardrails constrain the agent's decision-making process, preventing it from taking actions that could cause harm, violate policies, or exceed its authorized capabilities.
 - Resource guardrails limit the agent's access to systems, data, or computational resources, ensuring it can't overuse resources or access sensitive information it shouldn't.
 - Temporal guardrails implement timeouts, rate limits, and other time-based constraints to prevent runaway processes or excessive resource consumption.
 - Domain-specific guardrails restrict the agent to operate only within its intended domain or use case, preventing it from attempting tasks it's not designed for.
 
These guardrails are essential for deploying AI agents in real-world applications where safety, reliability, and adherence to ethical guidelines are crucial. They help ensure that agents remain helpful while avoiding potential risks or unintended consequences.

# OpenAI Agents SDK - Agent() Parameters Guide

This is a comprehensive list of all parameters you can pass to the `Agent()` constructor in the OpenAI Agents Python SDK.

## Core Parameters

### `name` (str, required)
- The name/identifier for the agent
- Used for identification in logs and tracing

### `instructions` (str or callable)
- Also known as a developer message or system prompt
- Can be a static string or a dynamic function that receives context and agent
- For dynamic instructions: `def dynamic_instructions(context: RunContextWrapper[UserContext], agent: Agent[UserContext]) -> str`

## Model Configuration

### `model` (str or Model instance)
- Which LLM to use
- Can be a string like `"o3-mini"`, `"gpt-4o"`, etc.
- Or a Model instance like `OpenAIChatCompletionsModel()`

### `model_settings` (ModelSettings)
- Optional model configuration parameters such as temperature
- Configure temperature, top_p, max_tokens, tool_choice, etc.
- Example: `ModelSettings(temperature=0.1, extra_args={"service_tier": "flex"})`

## Tool and Capability Configuration

### `tools` (list)
- Tools that the agent can use to achieve its tasks
- Can include function tools, hosted tools (WebSearchTool, FileSearchTool), etc.

### `handoffs` (list)
- Sub-agents that the agent can delegate to
- List of other Agent instances for delegation/routing

### `handoff_description` (str)
- Provide additional context for determining handoff routing
- Description used when this agent is a handoff target

## Output Configuration

### `output_type` (type)
- If you want the agent to produce a particular type of output
- Enables structured outputs using Pydantic models, dataclasses, TypedDict, etc.
- When you pass an output_type, that tells the model to use structured outputs instead of regular plain text responses

## Advanced Features

### `hooks` (AgentHooks)
- You can hook into the agent lifecycle with the hooks property
- Subclass AgentHooks to observe agent lifecycle events

### `input_guardrails` (list)
- Guardrails allow you to run checks/validations on user input
- List of input validation functions

### `output_guardrails` (list)
- Similar to input guardrails but for output validation

## Behavior Configuration

### `tool_use_behavior` (str)
- If you want the Agent to completely stop after a tool call (rather than continuing with auto mode), you can set `Agent.tool_use_behavior="stop_on_first_tool"`

### `reset_tool_choice` (bool)
- This behavior is configurable via agent.reset_tool_choice
- Controls whether tool_choice resets to "auto" after tool calls

## MCP Integration (if using MCP extension)

### `mcp_servers` (list) - *Available with MCP extension*
- Specify which MCP servers to use
- List of MCP server names defined in configuration

## Example Usage

```python
from agents import Agent, ModelSettings, function_tool
from pydantic import BaseModel

class OutputFormat(BaseModel):
    result: str
    confidence: float

@function_tool
def search_tool(query: str) -> str:
    return f"Results for {query}"

agent = Agent(
    name="Research Assistant",
    instructions="You are a helpful research assistant",
    model="gpt-4o",
    model_settings=ModelSettings(temperature=0.1),
    tools=[search_tool],
    output_type=OutputFormat,
    handoff_description="Specialist for research tasks"
)
```

## Quick Reference Table

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `name` | str | ✅ | Agent identifier |
| `instructions` | str/callable | ✅ | System prompt/instructions |
| `model` | str/Model | ❌ | LLM model to use |
| `model_settings` | ModelSettings | ❌ | Model configuration |
| `tools` | list | ❌ | Available tools |
| `handoffs` | list | ❌ | Sub-agents for delegation |
| `handoff_description` | str | ❌ | Description for routing |
| `output_type` | type | ❌ | Structured output format |
| `hooks` | AgentHooks | ❌ | Lifecycle event hooks |
| `input_guardrails` | list | ❌ | Input validation |
| `output_guardrails` | list | ❌ | Output validation |
| `tool_use_behavior` | str | ❌ | Tool usage behavior |
| `reset_tool_choice` | bool | ❌ | Tool choice reset behavior |
| `mcp_servers` | list | ❌ | MCP servers (extension) |

## Notes

- Parameters marked with ✅ are required
- Parameters marked with ❌ are optional
- The `mcp_servers` parameter is only available when using the MCP extension package
- Dynamic instructions and guardrails support both sync and async functionsm


In [40]:
# # Adding guardrails
# from pydantic import BaseModel
# class NameCheckOutput(BaseModel):
#     is_name_in_message : bool
#     name : str

# guardrail_agent = Agent(
#     name='Name-checker',
#     instructions="Check if the user is including someone's personal name in what they want you to do.",
#     model = 'gpt-4o-mini',
#     # tools= [],
#     # handoffs = [],
#     # handoff_description = '',
#     output_type = NameCheckOutput
# )

**What is ctx in this guardrail function?**
ctx is a context object that:

Contains execution state: It holds information about the current request, environment, and execution state
Provides access to context information: Note how it's used in context=ctx.context to pass contextual information to the Runner.run function
Is automatically injected: The framework automatically provides this parameter when your guardrail function is called

**What does ctx typically contain?**
While the exact contents depend on the framework you're using, a context object like ctx typically includes:

Request metadata: Information about the current request
Conversation history: Previous messages or interactions
User information: Details about the user making the request (if authenticated)
System state: Current state of the application or agent system
Configuration settings: Runtime configuration that might affect guardrail behavior

**How is ctx being used in your code?**
In your specific example:

ctx.context is being passed to Runner.run to provide the necessary context for the guardrail agent to properly evaluate the message
This ensures the guardrail agent has access to the same contextual information as the main agent

This pattern allows your guardrail to make informed decisions based not just on the current message, but potentially on the broader context of the conversation or application state.

In [41]:
class NameCheckOutput(BaseModel):
    is_name_in_message : bool
    name : str

guardrail_agent = Agent(
    name = 'Name-checker',
    instructions = "Check if the user is including someone's personal name in what they want you to do.",
    model = "gpt-4o-mini",
    output_type = NameCheckOutput
)

# What is GuardrailFunctionOutput?
GuardrailFunctionOutput is a structured return type specifically designed for guardrail functions. It serves as a standardized way to communicate the results of guardrail checks back to the system.
Purpose of GuardrailFunctionOutput
This class provides a consistent interface for guardrails to:

Signal whether a tripwire was triggered: The tripwire_triggered parameter indicates if the guardrail detected a condition that requires intervention
Return structured metadata: The output_info parameter allows passing additional information about what was detected
Enable standardized handling: By using a consistent return type, the framework can process guardrail results uniformly

Components of GuardrailFunctionOutput in Your Example
In your specific implementation:

tripwire_triggered:

Set to the boolean value of is_name_in_message
When True, signals that a personal name was detected in the message
This likely affects how the system processes the message further


output_info:

Contains a dictionary with the key "found_name"
Stores the full output from the guardrail agent (result.final_output)
This may include additional details like the actual name found and its context



How the System Uses GuardrailFunctionOutput
When your guardrail function returns this object:

If tripwire_triggered is True:

The system may block the request
It might request additional verification
It could modify how the message is processed
It might log the incident for review


The output_info can be:

Stored for audit purposes
Used to provide specific feedback to users
Passed to subsequent processing steps
Used for analytics and guardrail improvement



This standardized output structure enables a clean separation between detecting potentially problematic inputs and deciding how to handle them, giving the system flexibility in its response strategies.
</artifact>

In [58]:
from agents import GuardrailFunctionOutput, input_guardrail

@input_guardrail
async def guardrail_against_name(ctx,agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    print(is_name_in_message)
    print(result.final_output.name)
    return GuardrailFunctionOutput(output_info= {'Found Name' : result.final_output.name}, tripwire_triggered=is_name_in_message)

In [59]:
careful_sales_manager_agent = Agent(name = "Sales Manager",
instructions = sales_manager_instructions,
model = "gpt-4o-mini",
tools = content_tools,
handoffs = handoffs_for_sales_manager,
input_guardrails = [guardrail_against_name]
)


In [62]:
with trace("Protected SDK with guardrail for name"):
    result = await Runner.run(careful_sales_manager_agent, "Write and send a cold email to Dear CEO of Legtis.com ")

False
CEO of Legtis.com
Email sent with status code: 202
